# ASPrin experiments with the Dreyers dataset

To be used the `asprin` optimiser tool must be installed, and the easiest way to do it is via its [Anaconda package](https://anaconda.org/potassco/asprin).

The *Dreyers* dataset is available from the [tslaats/EventLogs](https://github.com/tslaats/EventLogs/tree/10db2d389b8190651bda5be578a7dad8123df384) repository. The [data/dreyers/fetch_dreyers_dataset.py](data/dreyers/fetch_dreyers_dataset.py) script downloads a local copy of the dataset and splits it in positive/negative log files.

In [ ]:
%run data/dreyers/fetch_dreyers_dataset.py

In [ ]:
from pathlib import Path

import negdis

Using `cerv` dataset as an example:

In [ ]:
DATA_DIR = Path('data', 'dreyers')

POS_LOG_PATH = DATA_DIR.joinpath('Dreyers Foundation.pos.xes')
NEG_LOG_PATH = DATA_DIR.joinpath('Dreyers Foundation.neg.xes')

DECLARE_RULES_PATH = DATA_DIR.joinpath('declare_rules.txt')

Input files can also be created on the fly, in this case the set of Declare patterns are defined directly in this notebook as a string. See below on how to use it (note the use of `r''` Python construct to avoid string interpolation). Strings can be passed as file arguments by using the `negdis.as_file` context function (see below for its usage).

In [ ]:
DECLARE_PATTERNS_STR = r'''
Absence(a):[^a]*
Absence2(a):[^a]*(a)?[^a]*
Absence3(a):[^a]*((a)?[^a]*){2}
AlternatePrecedence(a,b):[^b]*(a[^b]*b[^b]*)*[^b]*
AlternateResponse(a,b):[^a]*(a[^a]*b[^a]*)*[^a]*
AlternateSuccession(a,b):[^ab]*(a[^ab]*b[^ab]*)*[^ab]*
ChainPrecedence(a,b):[^b]*(ab[^b]*)*[^b]*
ChainResponse(a,b):[^a]*(ab[^a]*)*[^a]*
ChainSuccession(a,b):[^ab]*(ab[^ab]*)*[^ab]*
Choice(a,b):.*[ab].*
CoExistence(a,b):[^ab]*((a.*b.*)|(b.*a.*))*[^ab]*
End(a):.*a
Exactly1(a):[^a]*a[^a]*
Exactly2(a):[^a]*(a[^a]*){2}
ExclusiveChoice(a,b):([^b]*a[^b]*)|([^a]*b[^a]*)
Existence(a):.*a.*
Existence2(a):.*(a.*){2}
Existence3(a):.*(a.*){3}
Init(a):a.*
NotChainSuccession(a,b):[^a]*(aa*[^ab][^a]*)*([^a]*|aa*)
NotCoExistence(a,b):[^ab]*((a[^b]*)|(b[^a]*))?
NotSuccession(a,b):[^a]*(a[^b]*)*[^ab]*
Precedence(a,b):[^b]*(a.*b)*[^b]*
RespondedExistence(a,b):[^a]*((a.*b.*)|(b.*a.*))*[^a]*
Response(a,b):[^a]*(a.*b)*[^a]*
Succession(a,b):[^ab]*(a.*b)*[^ab]*
'''

## Choices generation stage

Run `negdis` to generate the candidates, output is stored in a temporary file. *Negdis* executable is wrapped in the `negdis.Negdis` class; its `default` method returns an object using the file in the `dist` directory.

In [ ]:
negdis_exe = negdis.Negdis.default()
print(negdis_exe.version())

In [ ]:
import atexit
import tempfile

out_dir = Path(tempfile.mkdtemp())

@atexit.register
def cleanup():
    import shutil
    # remove the temporary directory on termination
    shutil.rmtree(out_dir)


# output on a temporary file
CHOICES_PATH = Path(out_dir).joinpath('choices.json')

with negdis.as_file(DECLARE_PATTERNS_STR) as patterns:
    negdis_exe.discover(POS_LOG_PATH, NEG_LOG_PATH, patterns, CHOICES_PATH)

print(f'Choices written to: {CHOICES_PATH}')

### Shows top constraints

In [ ]:
negdis.count_choices(CHOICES_PATH)

## Optimisation stage

The configuration of the solver can be customised using the class `negdis.SolverConf`:

The configuration of the solver is customised using the class `negdis.SolverConf`:

```python
negdis.SolverConf.from_dict({
    'id': 'asprinminclos',
    'inputs': ['guess.lp'],
    'args': ['--quiet=1'],
    'docstring': 'Minimal cardinality wrt the closure constraints, and selected constraints for ties',
    'solver': 'asprin',
    'template': asprin_opt_code
})
```

Actual ASP program is generated by concatenating the content of files in `inputs` with the string `template`, and then replacing the values of `${...}` macros. Default macros are:

- `${predicate_action}(action)`: predicate with all the actions
- `${predicate_choice}(trace, constraint)`: the candidates for each trace
- `${predicate_constraint_action}(constraint, action)`: actions which are argument of the constraint
- `${predicate_constraint_name}(constraint, name, arity)`: name of the pattern of the constraint
- `${predicate_constraint}(constraint)`: all constraints that can be generated by the rules, and the candidates
- `${predicate_holds}(constraint)`: constraints deduced by the rules starting from the selected
- `${predicate_selected}(constraint)`: constraints selected for a specific model

each `constraint` is encoded as the function term `${functor_declare}(pattern, actions+)`. All these predicates except `${predicate_selected}` and `${predicate_holds}` are completely evaluated during grounding, so they can be used as *domain predicates*. 

New macros can be defined (or the default value replaced) by means of a dictionary that can be passed as an argument.

If the file names in `inputs` are not absolute paths, they'll be searched in the current directory followed by the directory of the Python package `negdis.templates`; e.g. the `guess.lp` is fetched from `negdis/templates/guess.lp` within one of the directories in `sys.path`. The location of the `negdis.templates` module directory can be verified using the following code:



The ASP program generated by a configuration (without the parts due to the specific problem) can be inspected using the `SolverConf.program` method, which accepts an *optional* argument with the mapping for the macros.

To show the original ASP program without the variable substitution the `SolverConf.program` method takes an optional `eval` argument which can be set to `False`:

All default solver configurations are available via the `negdis.configurations` function.

The `negdis.optimise_choices` function runs the optimisation code (see below for examples). The function takes an optional `mapping` argument which is used to expand the program.

Note that *asprin* solver doesn't support JSON output, so for the time being only the raw solver output is printed and no statistics are reported.

The configuration below uses the solver to optimise by preferring models that include a specific action. The name can be selected by setting the value for the `good_action` macro.

In [ ]:
opt_code = r'''
%%%%%%%%%%%%%%
%%%%%%% asprin optimisation statements

in_model(A) :- ${predicate_selected}(C), ${predicate_constraint_action}(C, A).

nice_model :- in_model(${good_action}).
not_nice_model :- not nice_model. 

#preference(p1,aso){ nice_model >> not_nice_model }.
#preference(p2,subset){ ${predicate_holds}(C) : ${predicate_constraint}(C) }.

#preference(p10,lexico){ 1::**p2; 2::**p1 }.
#optimize(p10).

#show in_model/1.
#show nice_model/0.
#show not_nice_model/0.
'''

opt_mode_nice = negdis.SolverConf.from_dict({
    'id': 'nice_models',
    'inputs': ['guess.lp'],
    'args': ['--quiet=1'],
    'docstring': 'Models without a specific constraint, and (subset) closure for ties',
    'solver': 'asprin',
    'template': opt_code
})

In the following two cells below preferred models are those with *Architect Review* and *Indledende afvisning* (i.e. *Initial rejection*) respectively. In both cases the solver is able to find models with the preferred action.

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode_nice, DECLARE_RULES_PATH, models=5, timeout=60, mapping={'good_action': '"Architect Review"'})

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode_nice, DECLARE_RULES_PATH, models=5, timeout=60, mapping={'good_action': '"Indledende afvisning"'})

## Cleanup temporary files

In [ ]:
import shutil

shutil.rmtree(str(out_dir))